Now take your Keras skills and go build another neural network. Pick your data set, but it should be one of abstract types, possibly even nonnumeric, and use Keras to make five implementations of your network. Compare them both in computational complexity as well as in accuracy and given that tradeoff decide which one you like best.

Your dataset should be sufficiently large for a neural network to perform well (samples should really be in the thousands here) and try to pick something that takes advantage of neural networks’ ability to have both feature extraction and supervised capabilities, so don’t pick something with an easy to consume list of features already generated for you (though neural networks can still be useful in those contexts).

Note that if you want to use an unprocessed image dataset, scikit-image is a useful package for converting to importable numerics.

In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils

# Import the backend
from keras import backend as K

### This project will use the CIFAR10 small image classification data set from Keras. It is a dataset of 50,000 32x32 color training images labeled over 10 categories, and 10,000 test images.

In [3]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 1) (10000, 1)


In [5]:
# Images are 32 x 32 x 3 (RGB), so reshape to 50,000, 3072 and 10,000, 3072
x_train = x_train.reshape(50000, 3072)
x_test = x_test.reshape(10000, 3072)
# Splitting labels into ten categories
# Normalizing the x features
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

## Implementation 1: Multi Layer Perceptron

In [6]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(3072,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# Output shape should be equal to the number of classes (10)
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                196672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 201,482
Trainable params: 201,482
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(x_train, y_train, batch_size = 64, epochs=10)
score = model.evaluate(x_test, y_test, verbose=False)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

Epoch 1/10
50000/50000 [==============================] - 2s - loss: 2.0338 - acc: 0.2473     
Epoch 2/10
50000/50000 [==============================] - 2s - loss: 1.9197 - acc: 0.2948     
Epoch 3/10
50000/50000 [==============================] - 2s - loss: 1.8852 - acc: 0.3110     
Epoch 4/10
50000/50000 [==============================] - 2s - loss: 1.8664 - acc: 0.3203     
Epoch 5/10
50000/50000 [==============================] - 2s - loss: 1.8526 - acc: 0.3264     
Epoch 6/10
50000/50000 [==============================] - 2s - loss: 1.8422 - acc: 0.3305     
Epoch 7/10
50000/50000 [==============================] - 2s - loss: 1.8250 - acc: 0.3396     
Epoch 8/10
50000/50000 [==============================] - 2s - loss: 1.8078 - acc: 0.3456     
Epoch 9/10
50000/50000 [==============================] - 2s - loss: 1.7981 - acc: 0.3473     
Epoch 10/10
50000/50000 [==============================] - 2s - loss: 1.7909 - acc: 0.3531     
Test Loss:  1.72420504169
Test Accuracy:  0.3626


#### Analysis of Implementation 1:
   Our model did not perform well. It took around twenty seconds and only had an accuracy of 36.26%

### Implementation 2: Multi Layer Perceptron

Changing optimizer to Adadelta based on results from http://cs231n.github.io/neural-networks-3/. 

In [9]:
# Modifying the original MLP
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(3072,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# Output shape should be equal to the number of classes (10)
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                196672    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 201,482
Trainable params: 201,482
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(x_train, y_train, batch_size = 64, epochs=10)
score = model.evaluate(x_test, y_test, verbose=False)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

Epoch 1/10
50000/50000 [==============================] - 3s - loss: 1.9952 - acc: 0.2706     
Epoch 2/10
50000/50000 [==============================] - 3s - loss: 1.8309 - acc: 0.3372     
Epoch 3/10
50000/50000 [==============================] - 3s - loss: 1.7690 - acc: 0.3643     
Epoch 4/10
50000/50000 [==============================] - 2s - loss: 1.7309 - acc: 0.3811     
Epoch 5/10
50000/50000 [==============================] - 2s - loss: 1.7013 - acc: 0.3908     
Epoch 6/10
50000/50000 [==============================] - 3s - loss: 1.6734 - acc: 0.4033     
Epoch 7/10
50000/50000 [==============================] - 3s - loss: 1.6521 - acc: 0.4087     
Epoch 8/10
50000/50000 [==============================] - 2s - loss: 1.6340 - acc: 0.4167     
Epoch 9/10
50000/50000 [==============================] - 3s - loss: 1.6140 - acc: 0.4262     
Epoch 10/10
50000/50000 [==============================] - 3s - loss: 1.5985 - acc: 0.4323     
Test Loss:  1.62469896145
Test Accuracy:  0.4186


#### Analysis of Implementation 2:
Using the adadelta optimizer improved the accuracy to 41.86%

### Implementation 3: Multi Layer Perceptron
Using Adadelta optimizer and leaky relu activation. Leaky ReLU is explained here  https://stats.stackexchange.com/questions/115258/comprehensive-list-of-activation-functions-in-neural-networks-with-pros-cons.

In [13]:
# Modifying the original MLP
model = Sequential()

model.add(Dense(64, activation=keras.layers.LeakyReLU(), input_shape=(3072,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation=keras.layers.LeakyReLU()))
model.add(Dropout(0.1))
# Output shape should be equal to the number of classes (10)
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 64)                196672    
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                650       
Total params: 201,482
Trainable params: 201,482
Non-trainable params: 0
_________________________________________________________________


C:\Users\maken\Anaconda3\lib\site-packages\keras\activations.py:103: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  ).format(identifier=identifier.__class__.__name__))


In [14]:
history = model.fit(x_train, y_train, batch_size = 64, epochs=10)
score = model.evaluate(x_test, y_test, verbose=False)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

Epoch 1/10
50000/50000 [==============================] - 3s - loss: 1.9634 - acc: 0.2890     
Epoch 2/10
50000/50000 [==============================] - 3s - loss: 1.8186 - acc: 0.3501     
Epoch 3/10
50000/50000 [==============================] - 3s - loss: 1.7571 - acc: 0.3746     
Epoch 4/10
50000/50000 [==============================] - 3s - loss: 1.7144 - acc: 0.3935     
Epoch 5/10
50000/50000 [==============================] - 3s - loss: 1.6808 - acc: 0.4066     
Epoch 6/10
50000/50000 [==============================] - 3s - loss: 1.6567 - acc: 0.4130     
Epoch 7/10
50000/50000 [==============================] - 3s - loss: 1.6325 - acc: 0.4206     
Epoch 8/10
50000/50000 [==============================] - 3s - loss: 1.6181 - acc: 0.4292     
Epoch 9/10
50000/50000 [==============================] - 3s - loss: 1.5992 - acc: 0.4362     
Epoch 10/10
50000/50000 [==============================] - 3s - loss: 1.5815 - acc: 0.4397     
Test Loss:  1.54988387165
Test Accuracy:  0.4508


#### Analysis of Implementation 3:
Using adadelta optimizer and leaky ReLU activations increased the accuray to 45.08%.

### Implementation 4: Convolutional Neural Network

In [14]:
# input image dimensions, from our data
img_rows, img_cols = 32, 32
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Epoch 1/10
50000/50000 [==============================] - 131s - loss: 1.7523 - acc: 0.3699   
Epoch 2/10
50000/50000 [==============================] - 128s - loss: 1.3678 - acc: 0.5133   
Epoch 3/10
50000/50000 [==============================] - 128s - loss: 1.2131 - acc: 0.5737   
Epoch 4/10
50000/50000 [==============================] - 130s - loss: 1.1163 - acc: 0.6070   
Epoch 5/10
50000/50000 [==============================] - 132s - loss: 1.0486 - acc: 0.6316   
Epoch 6/10
50000/50000 [==============================] - 139s - loss: 0.9893 - acc: 0.6557   
Epoch 7/10
50000/50000 [==============================] - 133s - loss: 0.9459 - acc: 0.6717   
Epoch 8/10
50000/50000 [==============================] - 133s - loss: 0.9040 - acc: 0.6842   
Epoch 9/10
50000/50000 [==============================] - 128s - loss: 0.8696 - acc: 0.6979   
Epoch 10/10
50000/50000 [==============================] - 128s - loss: 

#### Analysis of Convolutional Neural Network
Implementing a CNN improved accuracy to 68.7%.

### Implementation 5: Hierarchical Recurrent Neural Network

In [8]:
# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 295s - loss: 2.0196 - acc: 0.2585 - val_loss: 1.8669 - val_acc: 0.3305
Epoch 2/3
50000/50000 [==============================] - 294s - loss: 1.8232 - acc: 0.3359 - val_loss: 1.7692 - val_acc: 0.3554
Epoch 3/3
50000/50000 [==============================] - 296s - loss: 1.7392 - acc: 0.3674 - val_loss: 1.7275 - val_acc: 0.3676
Test loss: 1.72748284378
Test accuracy: 0.3676


#### Analysis of Implementation 5:
The Hierarchical Recurrent Neural Network implementation took about five minutes per epoch and resulted in an accuracy of 36.76%.

### Final Results:

Original Multi-Layer Perceptron: 2 seconds/epoch, 36.26% accuracy.

MLP Implementation 2: 2-3 seconds/epoch, 31.86% accuracy.

MLP Implementation 3: 2-3 seconds/epoch, 45.08% accuracy.

CNN Implementation: 130 seconds/epoch, 68.7% accuracy.

RNN Implementation: 295 seconds/epoch, 36.76% accuracy.

#### Conclusion:

The CNN Implementation returned the best accuracy with a large increase in computational time. The results were compared against a baseline of randomly choosing the category (10%). 